## K-NN Classifier

In [8]:
from init import *

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import datasets
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import pickle
import time

LOCATION = "pickles"

In [13]:
X = train_features.reshape(50000, 3*32*32)
Xt = test_features.reshape(10000, 3*32*32)
y = train_labels.flatten()
yt = test_labels.flatten()

In [14]:
msg("[K-NN] Training")
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X, y)
done()

pydump(knn, "pickles/knn_model.pickle")

[K-NN] Training ... done.


In [ ]:
msg("[K-NN] Testing Accuracy")

count = 0
start = time.time()
def predict_batch(x):
    global count
    global start
    count += 25
    elapsed = time.time()-start
    print("\rClassifiying {}. Elapsed Time: {}m {}s ... ".format(count, int(elapsed/60), int(elapsed%60)), end="")
    return knn.predict_proba(x)

Xt_pred_proba = np.array([predict_batch(Xt[i*25:(i+1)*25]) for i in range(400)]).reshape(-1, 10)
# Xt_pred = np.apply_along_axis(predict_with_update, 1, Xt).reshape(-1, 1)
# Xt_pred = knn.predict(Xt)

pydump(Xt_pred_proba, "pickles/knn_predict_proba.pickle")

msg(metrics.accuracy_score(yt, Xt_pred_proba.argmax(axis=1).reshape(-1,1)))
done()

## K-NN w/ PCA

In [5]:
X_pca = pyload("pickles/data_X_pca_200.pickle")
Xt_pca = pyload("pickles/data_Xt_pca_200.pickle")

In [8]:
msg("[K-NN PCA] Training")
knn_pca = KNeighborsClassifier(n_neighbors=10)
knn_pca.fit(X_pca, y)
done()

pydump(knn_pca, "pickles/knn_pca_model.pickle")

[K-NN PCA] Training ... done.


In [10]:
msg("[K-NN PCA] Testing Accuracy")

count = 0
start = time.time()
def predict_batch(x):
    global count
    global start
    count += 25
    elapsed = time.time()-start
    print("\rClassifiying {}. Elapsed Time: {}m {}s ... ".format(count, int(elapsed/60), int(elapsed%60)), end="")
    return knn_pca.predict_proba(x)

Xt_pca_pred_proba = np.array([predict_batch(Xt_pca[i*25:(i+1)*25]) for i in range(400)]).reshape(-1, 10)
# Xt_pred = np.apply_along_axis(predict_with_update, 1, Xt).reshape(-1, 1)
# Xt_pred = knn.predict(Xt)

pydump(Xt_pca_pred_proba, "pickles/knn_pca_predict_proba.pickle")

msg(metrics.accuracy_score(yt, Xt_pca_pred_proba.argmax(axis=1).reshape(-1,1)))
done()

Classifiying 10000. Elapsed Time: 2m 9s ... 0.3654 ... done.


In [12]:
pca_100 = PCA(n_components=100)
pca_100.fit(X,y)

PCA(copy=True, n_components=100, whiten=False)

In [13]:
X_pca_100 = pca_100.transform(X)
Xt_pca_100 = pca_100.transform(Xt)

pydump(pca_100, "pickles/pca_100.pickle")
pydump(X_pca_100, "pickles/X_pca_100.pickle")
pydump(Xt_pca_100, "pickles/Xt_pca_100.pickle")

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


### PCA / 100

In [5]:
def KNN_PCA(c=100):
    msg("[K-NN PCA {}] Preparing PCA ...".format(c))
    pca_100 = PCA(n_components=c)
    pca_100.fit(X,y)
    
    X_pca_100 = pca_100.transform(X)
    Xt_pca_100 = pca_100.transform(Xt)

    pydump(pca_100, "{}/pca_{}.pickle".format(LOCATION, c))
    pydump(X_pca_100, "{}/X_pca_{}.pickle".format(LOCATION, c))
    pydump(Xt_pca_100, "{}/Xt_pca_{}.pickle".format(LOCATION, c))
    
    msg("[K-NN PCA {}] Training".format(c))
    knn_pca_100 = KNeighborsClassifier(n_neighbors=10)
    knn_pca_100.fit(X_pca_100, y)
    done()
    
    pydump(knn_pca_100, "{}/knn_pca_{}_model.pickle".format(LOCATION, c))

    msg("[K-NN PCA {}] Testing Accuracy".format(c))
    
    global count
    global start
    count = 0
    start = time.time()
    def predict_batch(x):
        global count
        global start
        count += 25
        elapsed = time.time()-start
        print("\rClassifiying {}. Elapsed Time: {}m {}s ... ".format(count, int(elapsed/60), int(elapsed%60)), end="")
        return knn_pca_100.predict_proba(x)

    Xt_pca_pred_proba = np.array([predict_batch(Xt_pca_100[i*25:(i+1)*25]) for i in range(400)]).reshape(-1, 10)
    # Xt_pred = np.apply_along_axis(predict_with_update, 1, Xt).reshape(-1, 1)
    # Xt_pred = knn.predict(Xt)

    pydump(Xt_pca_pred_proba, "{}/knn_pca_{}_predict_proba.pickle".format(LOCATION, c))

    msg(metrics.accuracy_score(yt, Xt_pca_pred_proba.argmax(axis=1).reshape(-1,1)))
    done()

In [6]:
KNN_PCA(75)

[K-NN PCA 75] Preparing PCA ... ... [K-NN PCA 75] Training ... done.
Classifiying 10000. Elapsed Time: 1m 12s ... 0.3977 ... done.


In [7]:
KNN_PCA(50)

[K-NN PCA 50] Preparing PCA ... ... [K-NN PCA 50] Training ... done.
Classifiying 10000. Elapsed Time: 0m 48s ... 0.4012 ... done.


In [8]:
for i in [40,30,25,20,15,10,5,4,3,2,1]:
    KNN_PCA(i)

[K-NN PCA 40] Preparing PCA ... ... [K-NN PCA 40] Training ... done.
Classifiying 10000. Elapsed Time: 0m 38s ... 0.4093 ... done.
[K-NN PCA 30] Preparing PCA ... ... [K-NN PCA 30] Training ... done.
Classifiying 10000. Elapsed Time: 0m 27s ... 0.4178 ... done.
[K-NN PCA 25] Preparing PCA ... ... [K-NN PCA 25] Training ... done.
Classifiying 10000. Elapsed Time: 0m 23s ... 0.4157 ... done.
[K-NN PCA 20] Preparing PCA ... ... [K-NN PCA 20] Training ... done.
Classifiying 10000. Elapsed Time: 0m 18s ... 0.4084 ... done.
[K-NN PCA 15] Preparing PCA ... ... [K-NN PCA 15] Training ... done.
Classifiying 10000. Elapsed Time: 0m 10s ... 0.3875 ... done.
[K-NN PCA 10] Preparing PCA ... ... [K-NN PCA 10] Training ... done.
Classifiying 10000. Elapsed Time: 0m 4s ... 0.3493 ... done.
[K-NN PCA 5] Preparing PCA ... ... [K-NN PCA 5] Training ... done.
Classifiying 10000. Elapsed Time: 0m 0s ... 0.2671 ... done.
[K-NN PCA 4] Preparing PCA ... ... [K-NN PCA 4] Training ... done.
Classifiying 10000. 